# BaggingClassifier

In [1]:
# increase the width of the notebook
from IPython.display import display, HTML, Markdown

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
# Load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

y_train = train["Score"]
y_test = test["Score"]

X_train = train.drop("Score", axis=1)
X_test = test.drop("Score", axis=1)

## Transformations

In [4]:
from sklearn.pipeline import Pipeline

In [5]:
#Preprocessing pipelines
numeric_features = ["WhiteElo", "EloDif"]
categorical_features = ["Opening_name", "Time_format", "Increment_binary"]

numeric_transformer = Pipeline([
    ("scaler", StandardScaler())
])
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [6]:
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed  = preprocessor.transform(X_test)

## Train

In [7]:
dt = DecisionTreeClassifier(criterion='entropy', random_state=42)

In [8]:
bag_svm = BaggingClassifier(base_estimator=dt , random_state=42)

In [9]:
param_grid_bag = {
    'n_estimators': [100, 200],
    'max_samples':  [0.6,  1.0],       
    'max_features': [0.6,  1.0],
    'bootstrap':  [True, False]
}

In [10]:
#  Wrap in a GridSearchCV
grid_search_bag = GridSearchCV(
    bag_svm,
    param_grid_bag,
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
)

In [11]:
#  Fit on preprocessed training data
grid_search_bag.fit(X_train_transformed, y_train)

GridSearchCV(cv=3,
             estimator=BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                               random_state=42),
                                         random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_features': [0.6, 1.0],
                         'max_samples': [0.6, 1.0],
                         'n_estimators': [100, 200]},
             scoring='accuracy')

In [12]:
# Inspect the best settings
print("Best Bagging:", grid_search_bag.best_params_)
print("Best CV accuracy: {:.3f}".format(grid_search_bag.best_score_))

Best Bagging: {'bootstrap': True, 'max_features': 0.6, 'max_samples': 0.6, 'n_estimators': 200}
Best CV accuracy: 0.523


In [13]:
# Evaluate on test set
y_pred_bag = grid_search_bag.predict(X_test_transformed)
print("Test accuracy (Bagging): {:.3f}".format(accuracy_score(y_test, y_pred_bag)))
print("\nClassification Report:\n", classification_report(y_test, y_pred_bag))

Test accuracy (Bagging + SVM): 0.517

Classification Report:
               precision    recall  f1-score   support

   Black Win       0.50      0.47      0.48      4524
        Draw       0.00      0.00      0.00       566
   White Win       0.53      0.62      0.57      4910

    accuracy                           0.52     10000
   macro avg       0.34      0.36      0.35     10000
weighted avg       0.49      0.52      0.50     10000



C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\dimit\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### The Bagging Classifier achieved an accuracy of 0.517, which is the lowest performance we have observed across all the models tested.